## 심전도와 맥파를 이용한 딥러닝 기반 혈압 추정
---

### Code infomation

Created on Sun Aug 25 19:57:14 2019

Author : Kaintels (hanu)

Ver : 1.3.0

Environment : Anaconda 3 and Python 3.7.3 

### 데이터 세트 정보

1. 맥파(PPG) 신호 125Hz
2. 동맥혈압(ABP) 신호 125Hz
3. 심전도(ECG) 신호 125Hz

### 사용가능한 라이브러리

- numpy : 수치 및 행렬 계산을 위한 라이브러리
- matplotlib : 그래프 도식화 라이브러리
- pandas : 데이터 불러오기 및 데이터 처리 라이브러리
- scipy : 수학 및 신호처리 라이브러리
- tensorflow : 딥러닝 라이브러리
- keras : tensorflow의 high-level API

본 실습자료에는 cpu 버전만 사용 (gpu 버전에 비해 느림)

## 파이썬 라이브러리 import 및 예제 데이터 불러오기

---


### 라이브러리 설치 (!pip install -r requirements.txt을 실행, 실행은 run 또는 shift-enter, 창을 닫을시 자동 삭제됨)

In [1]:
!pip install -r requirements.txt # 처음 설치 필수, 종료시 매번 초기화되는 구조로 창을 닫을 시 자동삭제됨

ERROR: Invalid requirement: '#'


In [2]:
%config IPCompleter.greedy=True

In [3]:
!conda install graphviz

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
import keras

Using TensorFlow backend.


In [12]:
import tensorflow as tf
if type(tf.contrib) != type(tf): tf.contrib._warning = None

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(8, input_dim=4, activation="relu", kernel_initializer="uniform"))
model.add(Dense(6, activation="relu", kernel_initializer="uniform"))
model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))

In [13]:
from keras.utils import plot_model

In [14]:
from keras.utils import plot_model
plot_model(model)

In [16]:
from IPython.display import SVG
from keras.utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: cannot import name 'model_to_dot' from 'keras.utils' (C:\Users\hanwo\Miniconda3\envs\keras\lib\site-packages\keras\utils\__init__.py)